In [1]:
pip install PyMuPDF


  Obtaining dependency information for PyMuPDF from https://files.pythonhosted.org/packages/89/7c/e9d2259ec46547bfe4956fde5c4e42c1421acd20c4bcf2da772b4888bb6f/PyMuPDF-1.22.5-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 11.8/11.8 MB 28.4 MB/s eta 0:00:00


In [14]:
def es_pdf_editable(pdf_path):
    try:
        # Abre el archivo PDF
        doc = fitz.open(pdf_path)
        
        # Variable para almacenar el texto del PDF
        text = ""
        
        # Itera a través de las páginas del PDF y obtiene el texto
        for page in doc:
            text += page.getText()
        
        # Cierra el archivo PDF
        doc.close()
        
        # Verifica si el PDF contiene texto seleccionable
        return len(text.strip()) > 0
    except:
        # En caso de errores, asumimos que no es un PDF editable
        return False

def clasificar_pdf(pdf_path):
    if es_pdf_editable(pdf_path):
        return "PDF Editable"
    else:
        return "PDF Imagen"

# Ruta al archivo PDF a clasificar
archivo_pdf = "Music\Invoice_94202159.PDF"

# Clasifica el archivo PDF
resultado = clasificar_pdf(archivo_pdf)

# Imprime el resultado de la clasificación
print("El archivo es:", resultado)


El archivo es: PDF Imagen


In [30]:
import os
path = "pc\Music\hv Manrique andres.pdf"
os.system(path)

1